## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [3]:
import mysql.connector

cnx = mysql.connector.connect(host = "localhost",
user = "licascavalheiro",
password = "12345")


print(cnx)

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [4]:
cursor = cnx.cursor()

## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [ ]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [6]:
cursor.execute("SHOW DATABASES;")

for db in cursor:
    print(db)

('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [8]:
import pandas as pd
df_livros = pd.read_csv("/home/lucascavalheiro/pipeline-python-mongo-mysql/lista-produto.csv")
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,690a8c66fe046b52b55756ae,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,690a8c66fe046b52b55756af,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,690a8c66fe046b52b55756c1,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,690a8c66fe046b52b55756cf,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,690a8c66fe046b52b55756d1,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [9]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'Latitude', 'Longitude'],
      dtype='object')

In [10]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               
               PRIMARY KEY (id));
""")

In [11]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
  print(tb)

('tb_livros',)


**Selecionando a base de dados para verificar a tabela criada**

## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [12]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

[('690a8c66fe046b52b55756ae',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('690a8c66fe046b52b55756af',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('690a8c66fe046b52b55756c1',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('690a8c66fe046b52b55756cf',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('690a8c66fe046b52b55756d1',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('690a8c66fe046b52b55756d4',
  'Iniciando em progr

In [13]:
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

cursor.executemany(sql, lista_dados)
cnx.commit()

In [14]:
print(cursor.rowcount, "dados foram inseridos.")

742 dados foram inseridos.


In [15]:
cnx.close()


## Visualizando os dados inseridos